In [2]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.preprocessing.filters import Filters as filters
from ptrail.features.spatial_features import SpatialFeatures as spatial


# Reading the gulls dataset and converting to PTRAILDataFrame.
# Also, lets, print the first 5 points of the datset to
# see how the dataframe looks.
gulls = pd.read_csv('./data/geolife_sample.csv')
gulls = PTRAILDataFrame(gulls,
                        latitude='lat',
                        longitude='lon',
                        datetime='datetime',
                        traj_id='id',
                        rest_of_columns=[])
gulls.head()

lat         lon
traj_id DateTime                                  
1       2008-10-23 05:53:11  39.984224  116.319402
        2008-10-23 05:53:16  39.984211  116.319389
        2008-10-23 05:53:21  39.984217  116.319422
        2008-10-23 05:53:23  39.984710  116.319865
        2008-10-23 05:53:28  39.984674  116.319810

In [3]:
new_df = spatial.create_distance_between_consecutive_column(gulls)

In [4]:
filt_df = filters.hampel_outlier_detection(dataframe=new_df,
                                           column_name='Distance_prev_to_curr')

/home/yjharanwala/Desktop/PTRAIL/ptrail/preprocessing/filters.py:750: UserWarning: If kinematic features have been generated on the dataframe, then make sure to generate them again as outlier detection drops the point from the dataframe and does not run the kinematic features again.
  warnings.warn("If kinematic features have been generated on the dataframe, then make "


In [5]:
print(f"Length of original DF: {len(gulls)}")
print(f"Length of Filtered DF: {len(filt_df)}")
filt_df.head()

Length of original DF: 217653
Length of Filtered DF: 203019


lat         lon  Distance_prev_to_curr
traj_id DateTime                                                         
1       2008-10-23 05:53:11  39.984224  116.319402                    NaN
        2008-10-23 05:53:16  39.984211  116.319389               1.821083
        2008-10-23 05:53:21  39.984217  116.319422               2.889671
        2008-10-23 05:53:28  39.984674  116.319810               6.162987
        2008-10-23 05:53:33  39.984623  116.319773               6.488225